## Debbuging

In [ ]:
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'

In [ ]:
from flax.training import train_state
from flax.jax_utils import replicate
import jax
import optax
from jax.tree_util import tree_structure
import jax.numpy as jnp

from model import UnetJAX

In [ ]:
optimizer = optax.sgd(learning_rate=0.1, momentum=0.99)
unet = UnetJAX(input_image_size=512,
                use_activation=False, use_padding=True)
rng_key = jax.random.PRNGKey(0)
unet_params = unet.init_params(rng_key)

In [ ]:
state = train_state.TrainState.create(
    apply_fn=unet.apply,
    params=unet_params,
    tx=optimizer)

In [ ]:
tree_structure(state)

In [ ]:
jax.local_device_count()

In [ ]:
replicated_state = jax.tree_map(lambda x: jnp.array([x] * jax.local_device_count()), state)

In [ ]:
replicated_state

In [ ]:
using_replicate = replicate(state)
using_replicate

In [ ]:
print(type(replicated_state))
print(tree_structure(replicated_state))

In [ ]:
single_list = [1]
repl_arr = jax.tree_map(lambda x: jnp.array([x] * jax.local_device_count()), single_list)
repl_arr

In [ ]:
replicated_list = replicate(single_list)
replicated_list

In [ ]:
jnp.array([state]*jax.local_device_count())